In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Laden der Daten in ein Panda Data Frame

In [ ]:
df = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

# 2. Geschäftsverständnis

Welche Kombination von Attributen sorgen für eie hohe Weinqualität.

# 3. Datenverständnis (Exploratory Data Analysis)

# Beschreibung des Datensatzes

# Daten Visualisierung

In [ ]:
sns.countplot(x="quality", data=df)

In [ ]:
sns.pairplot(df, hue="quality")
plt.show()

Jetzt petrachten wir wie jede Variable Qualität einzeln beinflusst. https://stackoverflow.com/questions/31966494/compare-1-independent-vs-many-dependent-variables-using-seaborn-pairplot-in-an-h

In [ ]:
sns.pairplot(data=df, kind='reg', y_vars=['quality'], x_vars=[col for col in df.columns if col != 'quality'])

In [ ]:
sns.regplot(x="fixed acidity", y="quality", data=df)

# Untersuchug nach fehlenden Daten 
Quelle: https://towardsdatascience.com/how-to-check-for-missing-values-in-pandas-d2749e45a345

In [ ]:
df.isna().any().any()

Wir sehen, es fehlen keine Daten.

# 4. Datenaufbereitung


Für die Featureauswahl betrachten wir die Korrelation der Variblen von Quality.

In [ ]:
corr = df.corr()

In [ ]:
mask = np.triu(np.ones_like(corr, dtype=bool))

In [ ]:
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Wir löschen keine Features.

# Features
Die Features sind fixed acidit,volatile acidity	citric acid	residual sugar	chlorides	free sulfur dioxide	total sulfur dioxide	density	pH	sulphates	alcohol. Es sind objektive Eigenschafte eines Weines. Qualität ist kein Feature weil man es nicht wissenschaftlich messen kann. Unser Modell wird messen, wie die objektiven Features das subjektive Empfinden (also die Qualuität) beinflusst.



 #Daten Transformation

Zu Vereinfachung, wird die Qualität vonn Wein in einen Binären Wert Umgewandelt. Also gibt es Guten und Schlechten Wein. Wein mit Qualität von 3 bis 5 ist schlecht und wWein mit der Qualität von 6 bis 8 ist gut.

In [ ]:
df['quality'] = df['quality'].map(lambda x: x >= 6)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
sns.countplot(x="quality", data=df)

# Datenvorverarbeitung
y: Labels (Vektor)
X: Features (Matrix)
y_pred = f(X)



Datenaufteilung Train Test Split

In [ ]:
y = df['quality']

In [ ]:
class_names = list(y.unique())

In [ ]:
X = df.drop(columns='quality')
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)

In [ ]:
X_train

In [ ]:
y_train

# 5. Modellbildung

# K-Nächste Nachbarn Verfahren¶


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
y_pred_proba = clf.predict_proba(X_test)

KNN Evaluieren
Vergleichen:

y_test: Wahren Klassen (Labels)
y_pred: Vorhergesagte Klassen (Labels)

In [ ]:
from sklearn import metrics

In [ ]:
print("Accuracy-Score ist", metrics.accuracy_score(y_test, y_pred))

KNN hat einen niedrigen Accuracy Score und h

Cross Validation
Quelle: https://scikit-learn.org/stable/modules/cross_validation.html#computing-cross-validated-metrics

In [ ]:
 from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(clf, X, y, cv=5)

In [ ]:
scores

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
from sklearn import metrics

In [ ]:
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')

In [ ]:
scores